In [33]:
import pandas as pd
from refine_func import *


In [38]:
# load data
data = pd.read_csv("../data/kyoyang.csv")
data.reset_index(inplace=True)
data.head()

,index,year,semester,review,score
0,0,2024,1학기,"내 인생 최고의 꿀강. 그래서 강철이름만 보고 수강신청을 하였으나,, 다른 교수님도...",100
1,1,2024,여름,고딩 시절 사탐 내신 대비하듯이 하면 됨. 수업 잘 듣고 필기 정리한 다음 기출 구...,100
2,2,2024,여름,너희가 생각한 것보단 꿀강 아닐 가능성이 높다. 중기말 족보만 달달 외우면 된다. ...,100
3,3,2024,1학기,대 영 이 수강신청만 성공하면 에쁠 받으실 수 있습니다..,100
4,4,2024,1학기,꿀강 대 강철 초합금 강철 족보에서 대부분 출제하십니다. 적보만 통암기해가면 A+ ...,100


In [39]:
# 명사 불용어 처리
stopwords = ["이", "거", "게", "개", "심", "듯", "좀", "함", "것"]
# 전처리
# pandas의 review column -> python list[str]로
reviewList = data['review'].tolist()

# 명사 품사만 추출
tokens: list[list[str]] = extract_pos(reviewList, "Noun")

# 불용어 제외
tokens = filter_stopword(tokens, stopwords)

# 단순 참고용으로 top100 빈출 명사 출력
frequency: dict =  count_frequency(tokens)
sorted_frequency = list(sorted(frequency, key=frequency.get, reverse=True))
for token in sorted_frequency[:100]:
    print(f"{token}: {frequency[token]}")


수업: 5658
교수: 4346
시험: 4142
강의: 3458
과제: 2364
말: 2245
수: 2095
문제: 1768
내용: 1655
공부: 1550
중간: 1548
학점: 1455
성적: 1414
생각: 1382
점: 1279
시간: 1274
안: 1236
때: 1166
정도: 1121
정말: 1115
족보: 1071
학기: 1068
학생: 950
그냥: 918
점수: 901
번: 789
진짜: 763
추천: 751
저: 740
사람: 729
교양: 686
퀴즈: 651
하나: 645
출석: 600
설명: 579
부담: 576
분: 573
책: 542
때문: 535
전: 520
꿀강: 518
다른: 513
조금: 507
그: 481
이해: 480
매우: 479
발표: 472
중: 462
거의: 456
부분: 431
보고: 426
더: 425
질문: 418
만점: 410
오픈: 410
북: 410
주시: 409
제: 402
다만: 399
이번: 398
과목: 394
자체: 388
걸: 384
꼭: 377
개인: 376
대면: 375
필기: 370
본인: 366
보고서: 366
동영상: 358
임: 354
대해: 350
대한: 339
암기: 335
영어: 335
한번: 331
교재: 331
수강: 329
알: 328
느낌: 327
기준: 326
기말고사: 325
중간고사: 323
말씀: 315
진행: 312
관심: 311
별로: 310
무난: 309
건: 309
내: 306
주심: 306
처음: 306
제출: 302
평가: 298
피피티: 296
범위: 289
답: 287
후: 280
최고: 279
꽤: 279


In [44]:
# TfidVectorizeor가 문장을 input으로 받음
detokenized = deto(tokens)

# detokenized 된거 출력해보기
print(detokenized[:10])

['내 인생 최고 꿀강 강철 이름 보고 수강신청 다른 교수 사실 알 교수', '고딩 시절 사 내신 대비 듯이 수업 필기 정리 다음 기출 구해 회독 무난 변수 거의 안심', '너희 생각 꿀강 가능성 중 말 족보 달달 여기 족보 답 문제 포함 내용 빈칸 느낌 시험 기출 계 약간 변형 여름 문제 중 문제 변형 다만 난 기출 수업 기출 정답 선지 개념 숙지 기출 작품 교수 설명 내용 를 추천 출연 기출 개념 다른 부분 재 수 수업 벼락치기 당신 위해 추천 교수 수업 시간 강조 내용 시험 정말 계절 개꿀 위 관', '영 수강신청 에쁠 수', '꿀강 강철 초 합금 강철 족보 대부분 출제 적보 통 기해 가면 문제 올해 에쁠컷', '최고 강의 수업 꽤 호불호 갈릴 수 전자 출결 중간 강의 안 족보 말 족보 위주 이하 중간 겸 공부 만점 가까이 말 부담 훨', '그냥 무조건 강의 족보 보고 출석 수 강의 내용 꿀강 명강', '갓 말 말 뭐 교수', '데 이유 수업 시험 자체 족보 하루 전 공부 무리 학점 별개 영화 사람 이면 수 수업 임 출석 감점', '꿀강 꿀 강인 이유 사람 말']


In [46]:
# tf-idf 가져오기
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(detokenized)
print(f"shape {X.shape}")


shape (6068, 1000)


In [47]:
# LDA
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=6,learning_method='online',random_state=777,max_iter=3)
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
# print(lda_model.components_.shape) 


[[ 0.16692704  0.16723831  0.16728092 ...  0.16855311  0.22369211
   0.16901206]
 [ 0.16729773  0.17056714  0.1668648  ...  0.16765973  0.28797839
   0.16734929]
 [28.63520544 38.71878676  5.21872113 ... 27.38776971  4.86646159
   6.70730806]
 [ 0.16737492  0.16830383  0.16707033 ...  0.16841237  0.17056724
   0.17031618]
 [ 0.16705142  0.16959665  0.16744009 ...  0.1696372   0.18972058
   0.16992334]
 [ 0.17084737  0.16782471  0.16704525 ...  0.17050553  0.16775008
   8.18589516]]


In [49]:
terms = vectorizer.get_feature_names_out()

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], float(topic[i].round(2))) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)


Topic 1: [('테뱅', 20.43), ('문과', 18.85), ('쁠몰', 18.4), ('중국어', 16.73), ('편이', 11.96), ('강추', 10.5), ('동강', 9.48), ('여우', 9.15), ('커트라인', 8.11), ('노베', 7.84)]
Topic 2: [('수학', 42.89), ('통입', 31.8), ('재수강', 29.65), ('무조건', 28.74), ('회계', 28.37), ('독학', 26.3), ('에마', 15.72), ('족보', 15.67), ('빡센', 14.97), ('물리', 13.76)]
Topic 3: [('수업', 334.48), ('시험', 286.89), ('교수', 257.62), ('강의', 252.3), ('문제', 196.1), ('과제', 192.22), ('중간', 165.57), ('내용', 159.81), ('공부', 154.14), ('학점', 134.7)]
Topic 4: [('하라', 18.11), ('김숙경', 17.96), ('보장', 14.46), ('인품', 13.97), ('기량', 11.84), ('라틴어', 11.31), ('러시아', 10.79), ('편입', 9.45), ('통계학', 9.15), ('통입', 8.7)]
Topic 5: [('사랑', 28.15), ('어려움', 21.87), ('에쁠', 19.24), ('제발', 17.9), ('비쁠', 15.09), ('최악', 15.07), ('진심', 14.36), ('강철', 10.89), ('제공', 10.76), ('인간', 10.32)]
Topic 6: [('교수', 135.59), ('수업', 133.23), ('학점', 122.65), ('강의', 122.29), ('시험', 92.53), ('성적', 81.04), ('과제', 72.99), ('공부', 69.43), ('정말', 65.06), ('진짜', 51.06)]
